In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import geoplot
import matplotlib.colors as c
import geopandas as gpd
import os

# Make Animation for italy delinquency rate and loan amount change

In [ ]:
sns.set_theme(context='notebook', style='whitegrid', palette = 'deep')
dataset = pd.read_parquet('./loans_geography.parquet') 
dataset_bad_loans = pd.read_parquet('./bad_loans_rate.parquet')
data = gpd.read_file('./limits_IT_regions.geojson')
data['hue'] = 0
mapping = ['ITC1', 'ITC2', 'ITC4', 'ITHBI12', 'ITH3', 'ITH4', 'ITC3', 'ITH5', 'ITI1', 'ITI2', 'ITI3', 'ITI4', 'ITF1','ITF2', 'ITF3', 'ITF4', 'ITF5', 'ITF6', 'ITG1', 'ITG2']
dataset = 100*dataset.pct_change(6).dropna() 
dataset = dataset[mapping]
dataset_bad_loans = dataset_bad_loans[mapping]
dataset = dataset[::6]
dataset_bad_loans = 100*dataset_bad_loans.rolling(6, step=6).mean().dropna()
# dataset_bad_loans = dataset_bad_loans[6::6]
try : 
    os.mkdir('./images')
except : 
    pass

## Build animation images

In [8]:
for frame in range(len(dataset.index)):
    fig,axs = plt.subplots(1,2, figsize = (20,6))
    data['hue'] = dataset.iloc[frame,:].values
    data_plot =  data.explode(index_parts=False).reset_index(drop = True)
    axs[0].set_title(f'Delinquency rate {dataset.index[frame].strftime("%B %Y")}', y = -.1)
    axs[1].set_title(f'Loan amount pct change {dataset.index[frame].strftime("%B %Y")}', y = -.1)

    geoplot.choropleth(
                data_plot,
                hue = data_plot['hue'],
                norm = c.Normalize(-6,6),
                edgecolor='black',
                cmap = 'seismic',
                linewidth=.3,
                legend = True,
                ax = axs[1]
            )

    data['hue'] = dataset_bad_loans.iloc[frame,:].values
    data_plot =  data.explode(index_parts=False).reset_index(drop = True)
    geoplot.choropleth(
                data_plot,
                hue = data_plot['hue'],
                norm = c.Normalize(0,20),
                edgecolor='black',
                cmap = 'seismic',
                linewidth=.3,
                legend = True,
                ax = axs[0]
            )
    fig.savefig(f'./images/{frame}', dpi = 200, transparent = True)
    plt.close()
    # axs[1].set_title(dataset_bad_loans.index[frame].strftime("%B %Y"), y = -.1)

## Combine in gif format

In [9]:
images = []
for n in range(len(dataset.index)):
    exec('a'+str(n)+'=Image.open("./images/'+str(n)+'.png")')
    images.append(eval('a'+str(n)))
images[0].save('anim_full.gif',
               save_all=True,
               append_images=images[1:],
               disposal = 2,
               dpi = 200,
               duration = 600,
               loop=0)

